In [1]:
import pandas as pd
from pathlib import Path
from pandas.errors import EmptyDataError


def summarize_scenario_demo2(run_dir: Path, scenario: str) -> dict:
    dec = pd.read_csv(run_dir / scenario / "decision_log.csv")

    start_ab = dec.iloc[0]["allocation_id"]
    final_ab = dec.iloc[-1]["allocation_id"]

    fb_path = run_dir / scenario / "fallback_log.csv"

    # 기본값 (cold 대응)
    fallback_steps = []
    fallback_path = None
    fallback_count = 0

    core6_selected = []
    core6_alternatives = []
    core6_scores = []
    selection_sources = []

    if fb_path.exists():
        try:
            fb = pd.read_csv(fb_path)

            if len(fb) > 0:
                fallback_steps = fb["step"].tolist()
                fallback_count = len(fallback_steps)

                # A→B→C path
                fallback_path = (
                    fb[["previous_allocation", "fallback_allocation"]]
                    .astype(str)
                    .apply(lambda r: f"{r[0]}→{r[1]}", axis=1)
                    .tolist()
                )
                fallback_path = " → ".join(fallback_path)

                # Demo2 핵심
                core6_selected = fb["fallback_allocation"].astype(str).tolist()

                if "alternatives" in fb.columns:
                    core6_alternatives = (
                        fb["alternatives"]
                        .fillna("[]")
                        .apply(lambda x: x if isinstance(x, str) else "[]")
                        .tolist()
                    )

                if "fallback_score_n" in fb.columns:
                    core6_scores = fb["fallback_score_n"].tolist()

                if "selection_source" in fb.columns:
                    selection_sources = fb["selection_source"].tolist()

        except EmptyDataError:
            pass

    return {
        "scenario": scenario,
        "start_antibody": start_ab,
        "final_antibody": final_ab,
        "fallback_count": fallback_count,
        "fallback_steps": fallback_steps,
        "fallback_path": fallback_path,
        "core6_selected": core6_selected if core6_selected else None,
        "core6_alternatives": core6_alternatives if core6_alternatives else None,
        "core6_scores": core6_scores if core6_scores else None,
        "selection_sources": selection_sources if selection_sources else None,
    }

In [3]:
from pathlib import Path
import pandas as pd

RUN_DIR = Path("../artifacts/core11/core11_demo_002")

summary = []
for scenario in ["cold", "hot", "oscillation"]:
    summary.append(summarize_scenario_demo2(RUN_DIR, scenario))

summary_df = pd.DataFrame(summary)
summary_df

/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_17051/1402293106.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  .apply(lambda r: f"{r[0]}→{r[1]}", axis=1)
/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_17051/1402293106.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  .apply(lambda r: f"{r[0]}→{r[1]}", axis=1)


,scenario,start_antibody,final_antibody,fallback_count,fallback_steps,fallback_path,core6_selected,core6_alternatives,core6_scores,selection_sources
0,cold,A,A,0,[],None,None,None,None,None
1,hot,183,183,1,[0],A→183,[183],"[[""45"", ""1""]]",[0.0],[CORE6_WITH_CRITICAL]
2,oscillation,183,183,1,[0],A→183,[183],"[[""45"", ""1""]]",[0.0],[CORE6_WITH_CRITICAL]
